# Zero Value Point attack for RE

This (**bonus**) notebook explores the use of the ZVP attack for reverse-engineering.

 - You will first explore [symbolic execution](#Symbolic-formula-execution) of elliptic curve addition formulas.
 - Then you will [construct ZVP-points](#Constructing-ZVP-points) that lead to a zero intermediate value in a given formula.
 - Finally, you will use the points to [distinguish between the formulas](#Distinguishing-formulas).

In [ ]:
import re
from sympy import FF, ZZ, symbols, Poly

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters, get_params
from pyecsca.ec.point import Point
from pyecsca.ec.mod import mod, SymbolicMod
from pyecsca.ec.mult import LTRMultiplier, AccumulationOrder
from pyecsca.ec.context import local, DefaultContext
from pyecsca.misc.cfg import getconfig

First, lets setup some useful objects. We will be working with Jacobian coordinates on Short-Weierstrass curves, but other systems and curves would work as well.

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
which = "jacobian"
coords = model.coordinates[which]
secp256r1 = get_params("secg", "secp256r1", which)

def pp(poly):
    # Pretty printing a sympy Poly
    s = str(poly)[5:].split(",")[0]
    sup = {"0": "⁰", "1": "¹", "2": "²", "3": "³", "4": "⁴", "5": "⁵", "6": "⁶", "7": "⁷", "8": "⁸", "9": "⁹"}
    def repl(match):
        return "".join(sup[ch] for ch in match.group(1))
    return re.sub(r"\*\*([0-9]+)", repl, s).replace("*", " ")

## Symbolic formula execution

Now, lets use [sympy](https://www.sympy.org/en/index.html) polynomial arithmetic to execute a scalar multiplication symbolically, while keeping track of the intermediate values.

 - The following cell initializes a 64-bit prime order curve for demonstration purposes.
 - It then picks the `"add-2007-bl"` and `"dbl-2007-bl"` formulas and the LTR multiplier and symbolically executes a scalar multiplication `[5](X : Y : Z)`.
 - Finally, it prints the symbolic results of the execution.
 - The intermediates can be found in the `ctx` context.

In [ ]:
getconfig().ec.mod_implementation = "symbolic"
x, y, z = symbols("x y z")

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
field = FF(p)
a = SymbolicMod(Poly(0x4833d7aa73fa6694, x, y, z, domain=field), p)
b = SymbolicMod(Poly(0xa6c44a61c5323f6a, x, y, z, domain=field), p)
gx = SymbolicMod(Poly(0x5fd1f7d38d4f2333, x, y, z, domain=field), p)
gy = SymbolicMod(Poly(0x21f43957d7e20ceb, x, y, z, domain=field), p)
n = 0xc50de885003b80eb
h = 1

infty = Point(coords, X=mod(0, p), Y=mod(1, p), Z=mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)


add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
mult = LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqRP, True, True)


point = Point(coords,
              X=SymbolicMod(Poly(x, x, y, z, domain=field), params.curve.prime),
              Y=SymbolicMod(Poly(y, x, y, z, domain=field), params.curve.prime),
              Z=SymbolicMod(Poly(z, x, y, z, domain=field), params.curve.prime))
with local(DefaultContext()) as ctx:
    mult.init(params, point)
    res = mult.multiply(5)

x_poly = Poly(res.X.x, domain=field)
y_poly = Poly(res.Y.x, domain=field)
z_poly = Poly(res.Z.x, domain=field)
display(x_poly, y_poly, z_poly)

getconfig().ec.mod_implementation = "python"

The three output coordinates are polynomials in the coordinates of the input point `(X : Y : Z)`.

Let's now look at the intermediates in the `ctx` context.

 - The following cell prints the symbolic intermediate values as they appeared in the execution of the formulas.

In [ ]:
for formula_node in ctx.actions[0].children:
    print(formula_node.action.formula.shortname)
    for opres in formula_node.action.op_results:
        name = opres.name.ljust(5)
        output = "*" if opres.name in formula_node.action.formula.outputs else " "
        value = pp(opres.value)
        print(name, output, value[:120] + "..." if len(value) > 100 else value)
    print()

We can also unroll individual formulas with symbolic inputs `(X1 : Y1 : Z1)` and `(X2 : Y2 : Z2)`. This gives a list of polynomials in these input variables as well as in the curve parameters `a` and `b`. Here, we are unrolling two different addition formulas, one is the `add-2007-bl` as above, the other is `add-1986-cc`.

In [ ]:
from pyecsca.ec.formula.unroll import unroll_formula

one_add = add
other_add = coords.formulas["add-1986-cc"]

unrolled_one = unroll_formula(one_add)
print(one_add)
for (name, val), op in zip(unrolled_one, add.code):
    print(str(op).ljust(15), pp(val))
print("\n")

unrolled_other = unroll_formula(other_add)
print(other_add)
for (name, val), op in zip(unrolled_other, other_add.code):
    print(str(op).ljust(15), pp(val))

**Task**: Think about how you would try to distinguish between these two formulas?

## Constructing ZVP points

To distinguish between the two example formulas we will look at their intermediates as a set of polynomials. We will take this set, map it to affine coordinates, filter it and factor the polynomials into irreducible factors. The resulting set we will call the formula's *factor set*. The details of the filtering are too complicated, you can consult the [compute_factor_set](https://github.com/J08nY/pyecsca/blob/master/pyecsca/sca/re/zvp.py#L146-L195) function source for more. The gist of it is that we filter out polynomials that we cannot reliably force to zero. Note that after this step, very little polynomials remain.

In [ ]:
from pyecsca.sca.re.zvp import compute_factor_set

In [ ]:
fset_one = compute_factor_set(one_add)
for poly in fset_one:
    print(pp(poly))
print("------")
fset_other = compute_factor_set(other_add)
for poly in fset_other:
    print(pp(poly))

Let's take one polynomial that is in the factor set of the first formula but not in the second.

In [ ]:
diffs = fset_one.difference(fset_other)
print(diffs)
poly = diffs.pop()

This is the polynomial we want to be able to force to zero during a particular formula execution. Imagine a scalar multiplication with a known scalar (on a known curve) in which we control the input point. We can simulate the scalar multiplier and obtain the addition chain for a given scalar (i.e. a sequence of add/dbl formula applications with discrete log relationships between their input points).

Let's say the addition chain looks as follows:
```
dbl(P)
add(P, 2P)
dbl(2P)
add(P, 4P)
...
```

**Task**: How would you force a zero in a particular `add` formula application (i.e. come up with a point P)?

**Solution**:

For each `add()` call above we can try to construct a ZVP point that zeros the target polynomial when the given discrete log relationship between the points holds. The way we do this is quite simple: substitute in the multiplication-by-n (`n=4` for `add(P, 4P)`) map for the variables `x2`, `y2`, then eliminate the `y` variables using the curve equation, finally substitute the curve parameters and find roots of the given univariate polynomial. However, for many relationships there is going to be no such point. See the [A formula for disaster: a unified approach to elliptic curve special-point-based attacks](https://crocs.fi.muni.cz/public/papers/formulas_asiacrypt21) paper for more details of this construction (DCP problem solving).

The following cell constructs ZVP point(s) for the above polynomial, on the secp256r1 curve and the dlog relationship in `add(P, 4P)`.

In [ ]:
from pyecsca.sca.re.zvp import zvp_points

pts = zvp_points(poly, secp256r1.curve, 4, secp256r1.order)
print(pts)

## Distinguishing formulas

Let's now use these points for distinguishing between the two formulas. We will iterate over the two formulas and simulate the `add(P, 4P)` formula application with each formula, using the computed ZVP points for `P`. We will track the intermediate values and see whether they contain a zero.

**Task**: What do you expect to see with regards to the zero intermediate values in the two formulas?

In [ ]:
for formula in (one_add, other_add):
    for point in pts:
        print(formula, "input point", point)
        # Compute the [4]P to input into the formula
        multiple = secp256r1.curve.affine_multiply(point, 4)
        # Transform into Jacobian (randomized!)
        point_jacobian = point.to_model(coords, secp256r1.curve, randomized=True)
        multiple_jacobian = multiple.to_model(coords, secp256r1.curve, randomized=True)
        # Simulate the formula application while keeping track of intermediates
        with local(DefaultContext()) as fctx:
            result = formula(secp256r1.curve.prime, point_jacobian, multiple_jacobian, **secp256r1.curve.parameters)
        # Go over the intermediates and spot the zero.
        zero = False
        for opres in fctx.actions[0].action.op_results:
            zero |= opres.value == 0
            print(opres.name.ljust(5), "!!!" if opres.value == 0 else "   " , opres.value)
        print("zero observed" if zero else "no zero observed")
        print()

As you can see, we successfully introduced a zero intermediate value into the formula call `add-2007-bl(P, 4P)` but not into `add-1986-cc(P, 4P)`. Assuming we can detect the zero using a side-channel we can now distinguish the two formulas. Extending this to multiple formulas leads to the same overal RE strategy as in RPA, only the configurations, the inputs and the oracle used to build the decision table change. Consider the table below:

<img src="../img/zvp_table.svg" alt="drawing" width="100%" style="margin: auto"/>

Read [the paper](https://pyecsca.org/papers.html?utm_source=tutorial-ches2024#pyecsca-reverse-engineering-black-box-elliptic-curve-cryptography-via-side-channel-analysis) for more information on this and other RE techniques.